In [1]:
import os
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from scipy.optimize import curve_fit

from src.constants import (
	DIR_DATA,
	DIR_GRAPH,
	DIR_PICKLE,
	FILE_DATA_FRAME,
	SOLUTION_DP,
	SOLUTION_GREEDY,
	SOLUTION_CHRISTOFIDES,
	SOLUTION_SIMULATED_ANNEALING,
	SOLUTION_LIN_KERNIGHAN,
	SOLUTION_GENETIC,
	SOLUTION_ANT_COLONY,
	ALL_SOLUTIONS
)

from src.utils import (
	print_time_taken,
	process_all_files_to_png,
	process_all_files_to_dataframe,
	save_dataframe_to_file,
	load_dataframe_from_file
)

from src.solver.dynamic_programming_tsp_solver import DynamicProgrammingTspSolver
from src.solver.greedy_tsp_solver import GreedyTspSolver
from src.solver.christofides_tsp_solver import ChristofidesTspSolver
from src.solver.simulated_annealing_tsp_solver import SimulatedAnnealingTspSolver
from src.solver.lin_kernighan_tsp_solver import LinKernighanSolver
from src.solver.genetic_tsp_solver import GeneticAlgorithmSolver
from src.solver.ant_colony_tsp_solver import AntColonySolver

# Pre-Processing

## Creating Graphs for Visualization

In [2]:
# Create the output "graph" directory if it doesn't exist
if not os.path.exists(DIR_GRAPH):
	os.makedirs(DIR_GRAPH)

# Process all XML files in "data" directory and generate graphs
# process_all_files_to_png(DIR_DATA, DIR_GRAPH)

## Constructing DataFrame of Problem-Graphs

In [3]:
# Process all the files in the "data" directory into a DataFrame
# df = process_all_files_to_dataframe(DIR_DATA)

# Create the output "pickle" directory if it doesn't exist
if not os.path.exists(DIR_PICKLE):
	os.makedirs(DIR_PICKLE)

# Save the DataFrame to a "tsp_problems_dataframe.pkl" file
# save_dataframe_to_file(df, FILE_DATA_FRAME)

# Load the DataFrame from the file
df_loaded = load_dataframe_from_file(FILE_DATA_FRAME)

# Display the first few rows to confirm
df_loaded.head()

DataFrame loaded successfully.


,tsp_problem,number_of_vertices,graph,optimal_cost
0,ch130,130,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",6110.0
1,gr666,666,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",294358.0
2,kroB100,100,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",22141.0
3,d198,198,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",15780.0
4,kroA150,150,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",26524.0


# TSP Solutions

In [4]:
def solve_tsp(df, solver, algorithm_name):
	# Check if the new columns exists, if not, create them
	create_performance_metric_columns_if_not_exist(df, algorithm_name)

	num_of_problems = len(df.index)
	# Loop through each row in the dataframe and solve the TSP problem using TspSolver
	for index, row in df.iterrows():
		graph = row['graph']
		problem_name = row['tsp_problem']
		
		print(f"[{index + 1}/{num_of_problems}] Solving TSP using \"{algorithm_name}\" for problem: {problem_name} with {row['number_of_vertices']} vertices...")
		
		# Solve the TSP using the solver
		start = time.time()
		solution = solver.solve(graph)
		end = time.time()

		if solution:
			print_time_taken(f"solve TSP using \"{algorithm_name}\" for problem: {problem_name}", start, end)

			# Obtain/calculate performance metrics
			tour, total_cost = solution
			execution_time = end - start
			optimal_cost = row.get('optimal_cost', None)
			deviation_from_optimal = calculate_deviation_from_optimal_cost(total_cost, optimal_cost)

		else:
			print("Did not solve.")

			# Set performance metrics to n/a values
			tour = None
			total_cost = np.nan
			execution_time = np.nan
			optimal_cost = np.nan
			deviation_from_optimal = np.nan

		print("=====================================")

		# Update the dataframe
		update_data_frame_row_with_performance_metrics(df, index, algorithm_name, tour, total_cost, execution_time, deviation_from_optimal)

In [5]:
def create_performance_metric_columns_if_not_exist(df, algorithm_name):
	if f'{algorithm_name}_tour' not in df.columns:
		df[f'{algorithm_name}_tour'] = pd.Series(dtype='object')
	if f'{algorithm_name}_cost' not in df.columns:
		df[f'{algorithm_name}_cost'] = pd.Series(dtype='float')
	if f'{algorithm_name}_execution_time' not in df.columns:
		df[f'{algorithm_name}_execution_time'] = pd.Series(dtype='float')
	if f'{algorithm_name}_deviation_from_optimal' not in df.columns:
		df[f'{algorithm_name}_deviation_from_optimal'] = pd.Series(dtype='float')

In [6]:
def calculate_deviation_from_optimal_cost(total_cost, optimal_cost):
	if optimal_cost is not None:
		return (total_cost - optimal_cost) / optimal_cost * 100

In [7]:
def update_data_frame_row_with_performance_metrics(df, index, algorithm_name, tour, total_cost, execution_time, deviation_from_optimal):
	df.at[index, f'{algorithm_name}_tour'] = tour
	df.at[index, f'{algorithm_name}_cost'] = total_cost
	df.at[index, f'{algorithm_name}_execution_time'] = execution_time
	df.at[index, f'{algorithm_name}_deviation_from_optimal'] = deviation_from_optimal

## Exact Algorithms

### Held-Karp Algorithm (Dynamic Programming)

In [8]:
solve_tsp(df_loaded, DynamicProgrammingTspSolver(), SOLUTION_DP)

df_loaded.head()

[1/72] Solving TSP using "dp" for problem: ch130 with 130 vertices...
Did not solve.
[2/72] Solving TSP using "dp" for problem: gr666 with 666 vertices...
Did not solve.
[3/72] Solving TSP using "dp" for problem: kroB100 with 100 vertices...
Did not solve.
[4/72] Solving TSP using "dp" for problem: d198 with 198 vertices...
Did not solve.
[5/72] Solving TSP using "dp" for problem: kroA150 with 150 vertices...
Did not solve.
[6/72] Solving TSP using "dp" for problem: tsp225 with 225 vertices...
Did not solve.
[7/72] Solving TSP using "dp" for problem: fl417 with 417 vertices...
Did not solve.
[8/72] Solving TSP using "dp" for problem: brg180 with 180 vertices...
Did not solve.
[9/72] Solving TSP using "dp" for problem: brazil58 with 58 vertices...
Did not solve.
[10/72] Solving TSP using "dp" for problem: lin105 with 105 vertices...
Did not solve.
[11/72] Solving TSP using "dp" for problem: gil262 with 262 vertices...
Did not solve.
[12/72] Solving TSP using "dp" for problem: pcb442 wit

,tsp_problem,number_of_vertices,graph,optimal_cost,dp_tour,dp_cost,dp_execution_time,dp_deviation_from_optimal
0,ch130,130,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",6110.0,None,NaN,NaN,NaN
1,gr666,666,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",294358.0,None,NaN,NaN,NaN
2,kroB100,100,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",22141.0,None,NaN,NaN,NaN
3,d198,198,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",15780.0,None,NaN,NaN,NaN
4,kroA150,150,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",26524.0,None,NaN,NaN,NaN


## Approximation and Heuristic Algorithms

### Greedy TSP

In [9]:
solve_tsp(df_loaded, GreedyTspSolver(), SOLUTION_GREEDY)

df_loaded.head()

[1/72] Solving TSP using "greedy" for problem: ch130 with 130 vertices...
Time taken to solve TSP using "greedy" for problem: ch130: 0.00658s
[2/72] Solving TSP using "greedy" for problem: gr666 with 666 vertices...
Time taken to solve TSP using "greedy" for problem: gr666: 0.61820s
[3/72] Solving TSP using "greedy" for problem: kroB100 with 100 vertices...
Time taken to solve TSP using "greedy" for problem: kroB100: 0.00328s
[4/72] Solving TSP using "greedy" for problem: d198 with 198 vertices...
Time taken to solve TSP using "greedy" for problem: d198: 0.01791s
[5/72] Solving TSP using "greedy" for problem: kroA150 with 150 vertices...
Time taken to solve TSP using "greedy" for problem: kroA150: 0.00877s
[6/72] Solving TSP using "greedy" for problem: tsp225 with 225 vertices...
Time taken to solve TSP using "greedy" for problem: tsp225: 0.02555s
[7/72] Solving TSP using "greedy" for problem: fl417 with 417 vertices...
Time taken to solve TSP using "greedy" for problem: fl417: 0.14837

,tsp_problem,number_of_vertices,graph,optimal_cost,dp_tour,dp_cost,dp_execution_time,dp_deviation_from_optimal,greedy_tour,greedy_cost,greedy_execution_time,greedy_deviation_from_optimal
0,ch130,130,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",6110.0,None,NaN,NaN,NaN,"[0, 40, 38, 70, 129, 49, 1, 117, 79, 45, 19, 3...",7575.286292,0.006576,23.981772
1,gr666,666,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",294358.0,None,NaN,NaN,NaN,"[0, 309, 308, 307, 301, 300, 299, 298, 296, 29...",366962.000000,0.618201,24.665204
2,kroB100,100,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",22141.0,None,NaN,NaN,NaN,"[0, 94, 97, 11, 70, 26, 60, 34, 93, 56, 33, 6,...",29155.043704,0.003280,31.678983
3,d198,198,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",15780.0,None,NaN,NaN,NaN,"[0, 1, 6, 5, 2, 3, 4, 7, 8, 9, 10, 11, 12, 40,...",18620.073812,0.017906,17.997933
4,kroA150,150,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",26524.0,None,NaN,NaN,NaN,"[0, 129, 91, 7, 41, 121, 79, 30, 88, 132, 137,...",33609.866971,0.008766,26.714926


## Christofides Algorithm

In [10]:
solve_tsp(df_loaded, ChristofidesTspSolver(), SOLUTION_CHRISTOFIDES)

df_loaded.head()

[1/72] Solving TSP using "christofides" for problem: ch130 with 130 vertices...
Time taken to solve TSP using "christofides" for problem: ch130: 0.21114s
[2/72] Solving TSP using "christofides" for problem: gr666 with 666 vertices...
Time taken to solve TSP using "christofides" for problem: gr666: 23.55052s
[3/72] Solving TSP using "christofides" for problem: kroB100 with 100 vertices...
Time taken to solve TSP using "christofides" for problem: kroB100: 0.04566s
[4/72] Solving TSP using "christofides" for problem: d198 with 198 vertices...
Time taken to solve TSP using "christofides" for problem: d198: 0.75198s
[5/72] Solving TSP using "christofides" for problem: kroA150 with 150 vertices...
Time taken to solve TSP using "christofides" for problem: kroA150: 0.33111s
[6/72] Solving TSP using "christofides" for problem: tsp225 with 225 vertices...
Time taken to solve TSP using "christofides" for problem: tsp225: 0.56609s
[7/72] Solving TSP using "christofides" for problem: fl417 with 417

,tsp_problem,number_of_vertices,graph,optimal_cost,dp_tour,dp_cost,dp_execution_time,dp_deviation_from_optimal,greedy_tour,greedy_cost,greedy_execution_time,greedy_deviation_from_optimal,christofides_tour,christofides_cost,christofides_execution_time,christofides_deviation_from_optimal
0,ch130,130,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",6110.0,None,NaN,NaN,NaN,"[0, 40, 38, 70, 129, 49, 1, 117, 79, 45, 19, 3...",7575.286292,0.006576,23.981772,"[0, 48, 52, 119, 59, 50, 41, 43, 39, 46, 36, 9...",1.784470e+04,0.211144,192.057244
1,gr666,666,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",294358.0,None,NaN,NaN,NaN,"[0, 309, 308, 307, 301, 300, 299, 298, 296, 29...",366962.000000,0.618201,24.665204,"[0, 665, 107, 108, 109, 110, 111, 449, 461, 46...",1.750465e+06,23.550516,494.672134
2,kroB100,100,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",22141.0,None,NaN,NaN,NaN,"[0, 94, 97, 11, 70, 26, 60, 34, 93, 56, 33, 6,...",29155.043704,0.003280,31.678983,"[0, 89, 20, 16, 77, 12, 37, 19, 79, 29, 50, 47...",5.950318e+04,0.045662,168.746557
3,d198,198,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",15780.0,None,NaN,NaN,NaN,"[0, 1, 6, 5, 2, 3, 4, 7, 8, 9, 10, 11, 12, 40,...",18620.073812,0.017906,17.997933,"[0, 119, 123, 124, 168, 61, 52, 136, 143, 144,...",7.218672e+04,0.751976,357.457031
4,kroA150,150,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",26524.0,None,NaN,NaN,NaN,"[0, 129, 91, 7, 41, 121, 79, 30, 88, 132, 137,...",33609.866971,0.008766,26.714926,"[0, 70, 40, 65, 64, 123, 117, 127, 42, 96, 142...",1.203042e+05,0.331112,353.567357


## Simulated Annealing

In [11]:
solve_tsp(df_loaded, SimulatedAnnealingTspSolver(), SOLUTION_SIMULATED_ANNEALING)

df_loaded.head()

[1/72] Solving TSP using "sa" for problem: ch130 with 130 vertices...
Time taken to solve TSP using "sa" for problem: ch130: 0.08755s
[2/72] Solving TSP using "sa" for problem: gr666 with 666 vertices...
Time taken to solve TSP using "sa" for problem: gr666: 0.26529s
[3/72] Solving TSP using "sa" for problem: kroB100 with 100 vertices...
Time taken to solve TSP using "sa" for problem: kroB100: 0.03429s
[4/72] Solving TSP using "sa" for problem: d198 with 198 vertices...
Time taken to solve TSP using "sa" for problem: d198: 0.06924s
[5/72] Solving TSP using "sa" for problem: kroA150 with 150 vertices...
Time taken to solve TSP using "sa" for problem: kroA150: 0.05183s
[6/72] Solving TSP using "sa" for problem: tsp225 with 225 vertices...
Time taken to solve TSP using "sa" for problem: tsp225: 0.07899s
[7/72] Solving TSP using "sa" for problem: fl417 with 417 vertices...
Time taken to solve TSP using "sa" for problem: fl417: 0.15859s
[8/72] Solving TSP using "sa" for problem: brg180 with

,tsp_problem,number_of_vertices,graph,optimal_cost,dp_tour,dp_cost,dp_execution_time,dp_deviation_from_optimal,greedy_tour,greedy_cost,greedy_execution_time,greedy_deviation_from_optimal,christofides_tour,christofides_cost,christofides_execution_time,christofides_deviation_from_optimal,sa_tour,sa_cost,sa_execution_time,sa_deviation_from_optimal
0,ch130,130,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",6110.0,None,NaN,NaN,NaN,"[0, 40, 38, 70, 129, 49, 1, 117, 79, 45, 19, 3...",7575.286292,0.006576,23.981772,"[0, 48, 52, 119, 59, 50, 41, 43, 39, 46, 36, 9...",1.784470e+04,0.211144,192.057244,"[65, 125, 84, 27, 90, 56, 55, 117, 30, 18, 15,...",2.362042e+04,0.087551,286.586172
1,gr666,666,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",294358.0,None,NaN,NaN,NaN,"[0, 309, 308, 307, 301, 300, 299, 298, 296, 29...",366962.000000,0.618201,24.665204,"[0, 665, 107, 108, 109, 110, 111, 449, 461, 46...",1.750465e+06,23.550516,494.672134,"[393, 396, 530, 653, 647, 416, 310, 400, 231, ...",3.519361e+06,0.265293,1095.605691
2,kroB100,100,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",22141.0,None,NaN,NaN,NaN,"[0, 94, 97, 11, 70, 26, 60, 34, 93, 56, 33, 6,...",29155.043704,0.003280,31.678983,"[0, 89, 20, 16, 77, 12, 37, 19, 79, 29, 50, 47...",5.950318e+04,0.045662,168.746557,"[6, 19, 10, 90, 2, 98, 93, 35, 44, 47, 81, 32,...",6.474740e+04,0.034291,192.432125
3,d198,198,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",15780.0,None,NaN,NaN,NaN,"[0, 1, 6, 5, 2, 3, 4, 7, 8, 9, 10, 11, 12, 40,...",18620.073812,0.017906,17.997933,"[0, 119, 123, 124, 168, 61, 52, 136, 143, 144,...",7.218672e+04,0.751976,357.457031,"[131, 152, 147, 158, 89, 23, 21, 13, 3, 1, 41,...",8.280767e+04,0.069244,424.763458
4,kroA150,150,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",26524.0,None,NaN,NaN,NaN,"[0, 129, 91, 7, 41, 121, 79, 30, 88, 132, 137,...",33609.866971,0.008766,26.714926,"[0, 70, 40, 65, 64, 123, 117, 127, 42, 96, 142...",1.203042e+05,0.331112,353.567357,"[112, 9, 52, 133, 115, 4, 146, 56, 91, 55, 105...",1.159725e+05,0.051831,337.236057


## Lin-Kernighan Heuristic

In [12]:
# solve_tsp(df_loaded, LinKernighanSolver(), SOLUTION_LIN_KERNIGHAN)

# df_loaded.head()

## Genetic Algorithm

In [13]:
# solve_tsp(df_loaded, GeneticAlgorithmSolver(), SOLUTION_GENETIC)

# df_loaded.head()

## Ant Colony Optimization (ACO)

In [14]:
# solve_tsp(df_loaded, AntColonySolver(), SOLUTION_ANT_COLONY)

# df_loaded.head()

# Performance Analysis

## Summary Statistics

In [15]:
# Create the list of column names dynamically
metric_columns = []
for solution in ALL_SOLUTIONS:
	metric_columns.append(f'{solution}_cost')
	metric_columns.append(f'{solution}_execution_time')
	metric_columns.append(f'{solution}_deviation_from_optimal')

# Display summary statistics
df_loaded[metric_columns].describe()

,dp_cost,dp_execution_time,dp_deviation_from_optimal,greedy_cost,greedy_execution_time,greedy_deviation_from_optimal,christofides_cost,christofides_execution_time,christofides_deviation_from_optimal,sa_cost,sa_execution_time,sa_deviation_from_optimal
count,4.000000,4.000000,4.0,7.200000e+01,72.000000,72.000000,7.200000e+01,72.000000,72.000000,7.200000e+01,72.000000,72.000000
mean,3076.500000,3.788762,0.0,3.876467e+05,0.121134,33.035781,2.784668e+06,4.306075,440.456731,5.796343e+06,0.090331,634.907499
std,2862.257093,2.703037,0.0,2.898035e+06,0.292460,61.879420,2.218057e+07,11.666878,1207.993402,4.673931e+07,0.096024,1326.856133
min,39.000000,0.498868,0.0,9.200000e+01,0.000046,3.496388,1.570000e+02,0.000551,20.061233,4.600000e+01,0.006082,0.391213
25%,1573.500000,2.113508,0.0,8.039782e+03,0.002538,20.969953,1.497330e+04,0.030195,164.270783,1.722648e+04,0.031108,146.260160
50%,2704.000000,4.304398,0.0,2.690342e+04,0.008734,24.778315,8.036258e+04,0.186158,264.614001,9.939008e+04,0.052109,320.756283
75%,4207.000000,5.979652,0.0,5.849190e+04,0.060800,28.145511,2.228753e+05,1.869396,440.318647,3.358382e+05,0.109943,734.956855
max,6859.000000,6.047385,0.0,2.463096e+07,1.981584,533.846154,1.883642e+08,79.208555,10410.256410,3.968577e+08,0.489268,10768.717949


## Cost Analysis

In [16]:
# Prepare data for the plot
costs = ['optimal_cost', *(map(lambda solution: f'{solution}_cost', ALL_SOLUTIONS))]
legend = ['Optimal', *(map(lambda solution: solution.capitalize(), ALL_SOLUTIONS))]

# Create a new DataFrame to store the averaged results
averaged_data = df_loaded.groupby('number_of_vertices', as_index=False).mean(numeric_only=True)

# Create a Plotly figure
fig = go.Figure()

# Add traces for each solution's cost
for idx, cost in enumerate(costs):
	# Directly use the averaged cost values
	y_values = averaged_data[cost]

	# Create a scatter plot with lines connecting the points
	fig.add_trace(go.Scatter(
		x=averaged_data['number_of_vertices'],
		y=y_values,
		mode='lines+markers',
		name=legend[idx],
		line=dict(dash='dash' if 'Dp' in legend[idx] else 'solid'),  # Dashed line for DP
		marker=dict(size=8),
		connectgaps=True  # This option connects gaps in data points
	))

# Update layout
fig.update_layout(
	title='TSP Solution Costs vs. Optimal Solution Costs',
	xaxis_title='Number of Vertices',
	yaxis_title='Cost',
	legend_title='Solutions',
	hovermode='x unified',  # Show hover information for all traces at x
)

# Show the plot
fig.show()

## Time Complexity Analysis

In [21]:
def matplotlib_color_to_plotly(color):
	r, g, b, a = color
	return f'rgba({int(r * 255)}, {int(g * 255)}, {int(b * 255)}, {a})'

In [22]:
# Define model functions
def exponential_model(x, a, b):
	return a * np.exp(b * x)

def quadratic_model(x, a, b, c):
	return a * x**2 + b * x + c

def quadratic_log_model(x, a, b):
	return a * (x**2 * np.log(x)) + b

def cubic_model(x, a, b, c, d):
	return a * x**3 + b * x**2 + c * x + d

In [23]:
def fit_and_plot_curve_fit(x_valid, y_valid, algorithm_name, color, fig):
	plotly_color = matplotlib_color_to_plotly(color)  # Convert color

	if algorithm_name == SOLUTION_DP:  # Held-Karp (exponential)
		popt, _ = curve_fit(exponential_model, x_valid, y_valid, p0=[1, 0.1])
		x_fit = np.linspace(min(x_valid), max(x_valid), 100)
		y_fit = exponential_model(x_fit, *popt)

		fig.add_trace(go.Scatter(
			x=x_fit,
			y=y_fit,
			mode='lines',
			name=f'{algorithm_name.capitalize()} (Exp Fit)',
			legendgroup=algorithm_name,  # Group data and curve under one legend
			line=dict(color=plotly_color)  # Solid line
		))

	elif algorithm_name == SOLUTION_CHRISTOFIDES:  # Christofides (cubic)
		popt, _ = curve_fit(cubic_model, x_valid, y_valid)
		x_fit = np.linspace(min(x_valid), max(x_valid), 100)
		y_fit = cubic_model(x_fit, *popt)

		fig.add_trace(go.Scatter(
			x=x_fit,
			y=y_fit,
			mode='lines',
			name=f'{algorithm_name.capitalize()} (Cubic Fit)',
			legendgroup=algorithm_name,  # Group data and curve under one legend
			line=dict(color=plotly_color)  # Solid line
		))

	elif algorithm_name == SOLUTION_LIN_KERNIGHAN:  # Lin-Kernighan (N^2 log N)
		popt, _ = curve_fit(quadratic_log_model, x_valid, y_valid)
		x_fit = np.linspace(min(x_valid), max(x_valid), 100)
		y_fit = quadratic_log_model(x_fit, *popt)

		fig.add_trace(go.Scatter(
			x=x_fit,
			y=y_fit,
			mode='lines',
			name=f'{algorithm_name.capitalize()} (Quad Log Fit)',
			legendgroup=algorithm_name,  # Group data and curve under one legend
			line=dict(color=plotly_color)  # Solid line
		))

	else:  # Greedy or any other algorithm (quadratic)
		popt, _ = curve_fit(quadratic_model, x_valid, y_valid)
		x_fit = np.linspace(min(x_valid), max(x_valid), 100)
		y_fit = quadratic_model(x_fit, *popt)

		fig.add_trace(go.Scatter(
			x=x_fit,
			y=y_fit,
			mode='lines',
			name=f'{algorithm_name.capitalize()} (Quad Fit)',
			legendgroup=algorithm_name,  # Group data and curve under one legend
			line=dict(color=plotly_color)  # Solid line
		))

In [24]:
def plot_execution_times(df, solutions):
	execution_times = list(map(lambda solution: f'{solution}_execution_time', solutions))
	colors = plt.cm.viridis(np.linspace(0, 1, len(execution_times)))

	fig = go.Figure()

	for idx, execution_time in enumerate(execution_times):
		algorithm_name = execution_time.replace('_execution_time', '')
		mask = df[execution_time].notna()
		x_valid = df.loc[mask, 'number_of_vertices']
		y_valid = df.loc[mask, execution_time]

		# Convert Matplotlib color to Plotly RGB format
		plotly_color = matplotlib_color_to_plotly(colors[idx])

		# Scatter plot for the data points (hide legend for this trace)
		fig.add_trace(go.Scatter(
			x=x_valid,
			y=y_valid,
			mode='markers',
			name=f'{algorithm_name.capitalize()} (Data)',  # Common legend entry
			legendgroup=algorithm_name,  # Group with the curve
			showlegend=False,  # Hide individual data points from legend
			marker=dict(size=8, color=plotly_color)
		))

		# Fit and plot best-fit line based on the algorithm's complexity
		fit_and_plot_curve_fit(x_valid, y_valid, algorithm_name, colors[idx], fig)

	fig.update_layout(
		title='Execution Time vs. Number of Vertices for All Solutions',
		xaxis_title='Number of Vertices',
		yaxis_title='Execution Time (s)',
		legend_title='Solutions',
		hovermode='x unified'
	)

	fig.show()

In [25]:
plot_execution_times(df_loaded, ALL_SOLUTIONS)

## Deviation From Optimality

In [26]:
# Prepare data for the plot
deviations_from_optimal = list(map(lambda solution: f'{solution}_deviation_from_optimal', ALL_SOLUTIONS))
legend = list(map(lambda solution: solution.capitalize(), ALL_SOLUTIONS))

# Create a new DataFrame to store the averaged results
averaged_deviation_data = df_loaded.groupby('number_of_vertices', as_index=False).mean(numeric_only=True)

# Create a Plotly figure
fig = go.Figure()

# Add traces for each solution's deviation from optimal
for idx, deviation in enumerate(deviations_from_optimal):
	# Directly use the averaged deviation values
	y_values = averaged_deviation_data[deviation]

	# Create a scatter plot with lines connecting the points
	fig.add_trace(go.Scatter(
		x=averaged_deviation_data['number_of_vertices'],
		y=y_values,
		mode='lines+markers',
		name=legend[idx],
		line=dict(dash='dash' if 'Dp' in legend[idx] else 'solid'),  # Dashed line for DP
		marker=dict(size=8),
		connectgaps=True  # This option connects gaps in data points
	))

# Update layout
fig.update_layout(
	title='Deviation of TSP Solutions from Optimal',
	xaxis_title='Number of Vertices',
	yaxis_title='Deviation from Optimal (%)',
	legend_title='Solutions',
	hovermode='x unified',  # Show hover information for all traces at x
)

# Show the plot
fig.show()